In [5]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import datetime

from qiskit_finance.data_providers import RandomDataProvider
from qiskit.result import QuasiDistribution
from qiskit_aer.primitives import Sampler
from qiskit_algorithms import NumPyMinimumEigensolver, QAOA
from qiskit_algorithms.optimizers import COBYLA, SPSA, POWELL
from qiskit_finance.applications.optimization import PortfolioOptimization
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms.utils import algorithm_globals
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit.circuit import ParameterVector
from qiskit_aer import Aer



from qiskit_aer import AerSimulator
from qiskit import transpile

from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error, ReadoutError

from normalize_data import normalize_data

In [6]:
#Noise Model
noise_model = NoiseModel()

# Gate noise
noise_model.add_all_qubit_quantum_error(
    depolarizing_error(0.001, 1), ['rx', 'ry', 'rz']
)

noise_model.add_all_qubit_quantum_error(
    depolarizing_error(0.01, 2), ['cx']
)

# Readout noise
readout_error = ReadoutError([[0.98, 0.02],
                              [0.03, 0.97]])
noise_model.add_all_qubit_readout_error(readout_error)

In [35]:
seed = 1234
num_assets = 4
q=0.5
budget = 2
penalty = num_assets
p = 3
# Generate expected return and covariance matrix from (random) time-series
stocks = [("TICKER%s" % i) for i in range(num_assets)]
data = RandomDataProvider(
    tickers=stocks,
    start=datetime.datetime(2016, 1, 1),
    end=datetime.datetime(2016, 1, 30),
    seed=seed,
)
data.run()
mu = data.get_period_return_mean_vector()
sigma = data.get_period_return_covariance_matrix()
mu_norm, sigma_norm = normalize_data(mu,sigma)

portfolio = PortfolioOptimization(
    expected_returns=mu_norm, covariances=sigma_norm, risk_factor=q, budget=budget
)
qp = portfolio.to_quadratic_program()
converter = QuadraticProgramToQubo()
qubo = converter.convert(qp)
cobyla = COBYLA()
cobyla.set_options(maxiter=100)         
qaoa_mes = QAOA(sampler=Sampler(), optimizer=cobyla, reps=p)
qaoa = MinimumEigenOptimizer(qaoa_mes,penalty)
result = qaoa.solve(qp)




In [ ]:
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit.quantum_info import SparsePauliOp

from qiskit_aer.primitives import Sampler as AerSampler
from qiskit_algorithms import QAOA


# 1️⃣ Define a simple problem
qp = QuadraticProgram()
qp.binary_var('x')
qp.binary_var('y')
qp.minimize(linear={'x': 1, 'y': 1}, quadratic={('x','y'): -2})

# 2️⃣ Convert to QUBO
converter = QuadraticProgramToQubo()
qubo = converter.convert(qp)

# 3️⃣ Convert QUBO to Ising (Pauli operator)
hamiltonian, offset = qubo.to_ising()

# 4️⃣ Build a noise model
noise_model = NoiseModel()
error_1q = depolarizing_error(0.01, 1)
error_2q = depolarizing_error(0.02, 2)
noise_model.add_all_qubit_quantum_error(error_1q, ['x', 'h'])
noise_model.add_all_qubit_quantum_error(error_2q, ['cx'])

# 5️⃣ Setup noisy AerSimulator and Sampler
simulator = AerSimulator(noise_model=noise_model, shots=1024)
sampler = AerSampler(noise_model=noise_model)

# 6️⃣ Build QAOA
optimizer = COBYLA(maxiter=100)
qaoa = QAOA(sampler=sampler, optimizer=optimizer, reps=2)

# 7️⃣ Run QAOA
result = qaoa.compute_minimum_eigenvalue(operator=SparsePauliOp(hamiltonian))

print("Eigenvalue:", result.eigenvalue)
print("Offset:", offset)

# 8️⃣ Get the parameterized circuit ansatz
qc_ansatz = qaoa.ansatz
print(qc_ansatz.draw('text'))


TypeError: Sampler.__init__() got an unexpected keyword argument 'noise_model'

In [9]:
print(qc_ansatz.draw('mpl'))

Figure(1332.89x284.278)
